In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle


import numpy as np
#to do, i need to create a matrix of units x timebins x trials per recording per event... 

num_pc_for_decoding = 10

neural_data_pca = [[{} for _ in range(len(neural_data[0]))] for _ in range(len(neural_data))]
#nueral_data = neurons x timebins x TRIALS 

for n1 in range(len(neural_data)):
    for n2 in range(len(neural_data[0])):
        if not neural_data[n1][n2]:
            continue

        neural_data_pca[n1][n2] = {"reward": [], "fear": [], "conflict": [], "habituation": [], "iti": []}

        for n3 in range(neural_data[n1][n2]["reward"].shape[2]:
            reward_data = neural_data[n1][n2]["reward"][:, :, n3]
            selected_neurons = np.where(np.all(neuron2animalidx == [n1, n2], axis=1))[0]
            #so can only use neuron rows from mouse who did that trial
            pca_coeff = coef[selected_neurons, :num_pc_for_decoding]
            result = pca_coeff.T.dot(reward_data)
            neural_data_pca[n1][n2]["reward"].append(result)

        for n3 in range(neural_data[n1][n2]["fear"].shape[2]):
            fear_data = neural_data[n1][n2]["fear"][:, :, n3]
            selected_neurons = np.where(np.all(neuron2animalidx == [n1, n2], axis=1))[0]
            pca_coeff = coef[selected_neurons, :num_pc_for_decoding]
            result = pca_coeff.T.dot(fear_data)
            neural_data_pca[n1][n2]["fear"].append(result)

        for n3 in range(neural_data[n1][n2]["conflict"].shape[2]):
            conflict_data = neural_data[n1][n2]["conflict"][:, :, n3]
            selected_neurons = np.where(np.all(neuron2animalidx == [n1, n2], axis=1))[0]
            pca_coeff = coef[selected_neurons, :num_pc_for_decoding]
            result = pca_coeff.T.dot(conflict_data)
            neural_data_pca[n1][n2]["conflict"].append(result)

        habituation_data = neural_data[n1][n2]["habituation"]
        selected_neurons = np.where(np.all(neuron2animalidx == [n1, n2], axis=1))[0]
        pca_coeff = coef[selected_neurons, :num_pc_for_decoding]
        result = pca_coeff.T.dot(habituation_data)
        neural_data_pca[n1][n2]["habituation"] = result

        for n3 in range(neural_data[n1][n2]["iti"].shape[2]):
            iti_data = neural_data[n1][n2]["iti"][:, :, n3]
            selected_neurons = np.where(np.all(neuron2animalidx == [n1, n2], axis=1))[0]
            pca_coeff = coef[selected_neurons, :num_pc_for_decoding]
            result = pca_coeff.T.dot(iti_data)
            neural_data_pca[n1][n2]["iti"].append(result)

#pca calc = W (pcs x neurons) x D (neurons x timebins)
#PCA output = PCsx timebins
#neural_data_pca = { event: list(pcs x timebins)}

num_fold = 5
num_shuffle = 5

decode_trial_type_auc = [{} for _ in range(len(neural_data_pca[0]))]
decode_idx2trial_type = {1: "reward", 2: "fear", 3: "conflict"}
decode_idx = [1, 2, 3]
# iterating through n1 which is the length of the first dimension of the neural_data_pca
#potentially this is number of event types? or number of timebins 
for n1 in range(len(neural_data_pca[0])): #hmmm i dont actually n1 is 
    decode_trial_type_auc_local = {}
    # this also seems like number of events, but i guess i subset of interest events 
    for n2 in decode_idx:
    #this foor loop looks for 1: 2&3, or 2: 1&3, 3: 2&3
        for n3 in [x for x in decode_idx if x != n2]:
            data_temp_pos = []
            data_temp_neg = []

            for n4 in range(len(neural_data_pca)): # for for every trial?
                data_temp_pos.append(neural_data_pca[n4][n1][decode_idx2trial_type[n2]])
                data_temp_neg.append(neural_data_pca[n4][n1][decode_idx2trial_type[n3]])
                #so creating temp lists for later of the two event types currently being iterated?
                #n4 = timebins , and n1 = PCs?????

            data_temp_pos = np.stack(data_temp_pos, axis=2)
            data_temp_neg = np.stack(data_temp_neg, axis=2)
             # i think this is just turning a list into a multi dimensional numpy array

            num_trial_pos = data_temp_pos.shape[2]
            num_trial_neg = data_temp_neg.shape[2] # is a constant that reflects.... something

            data_temp_pos = data_temp_pos[:, :, np.random.permutation(num_trial_pos)] # basically shuffling the order of trials
            data_temp_neg = data_temp_neg[:, :, np.random.permutation(num_trial_neg)] # so that the model is not trained on conescutive trials 
            # so that the test set is not from one animal or one session

            auc_glm = np.zeros((num_fold, T)) # i do not know what T is - possibly it is also timebins? 

            auc_rf = np.zeros((num_fold, T)) # possibly it is total time bins? event no. X event length
            auc_glm_shuffle = np.zeros((num_fold, T, num_shuffle))
            auc_rf_shuffle = np.zeros((num_fold, T, num_shuffle))
            # so this is setting empty arrays for the actual stuff below 

            for n4 in range(num_fold):
                # concatenate data_temp_pos with data_temp_neg along third axis (aka event) 
                #but only take 1/num_fold of data for test 
                data_test = np.concatenate((data_temp_pos[:, :, n4 * num_trial_pos // num_fold:(n4 + 1) * num_trial_pos // num_fold],
                                            data_temp_neg[:, :, n4 * num_trial_neg // num_fold:(n4 + 1) * num_trial_neg // num_fold]), axis=2)
               # creating the labels for the test set such that pos = 1 and neg trials = 0
               # result is a 1d numpy array such as: 111100000000000
                label_test = np.concatenate((np.ones((n4 + 1) * num_trial_pos // num_fold - n4 * num_trial_pos // num_fold),
                                             np.zeros((n4 + 1) * num_trial_neg // num_fold - n4 * num_trial_neg // num_fold)))
                data_train = np.concatenate((data_temp_pos[:, :, np.setdiff1d(np.arange(num_trial_pos), np.arange(n4 * num_trial_pos // num_fold, (n4 + 1) * num_trial_pos // num_fold))],
                                             data_temp_neg[:, :, np.setdiff1d(np.arange(num_trial_neg), np.arange(n4 * num_trial_neg // num_fold, (n4 + 1) * num_trial_neg // num_fold))]), axis=2)
                label_train = np.concatenate((np.ones(num_trial_pos - (n4 + 1) * num_trial_pos // num_fold + n4 * num_trial_pos // num_fold),
                                             np.zeros(num_trial_neg - (n4 + 1) * num_trial_neg // num_fold + n4 * num_trial_neg // num_fold))
                # check sizes of matrices before i even predict 
                # for testing set fold and shuffle number to 1
                # STOP HERE FOR TESTING 
                for n5 in range(T):
                    model_glm = LogisticRegression()
                    #its possible i might beed to squeeze this like in matlab
                    model_glm.fit(data_train[:, n5, :].T, label_train)
                    pred_glm = model_glm.predict_proba(data_test[:, n5, :].T)
                    auc_glm[n4, n5] = roc_auc_score(label_test, pred_glm[:, 1])

                    model_rf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=50, random_state=0)
                    model_rf.fit(data_train[:, n5, :].T, label_train)
                    pred_rf = model_rf.predict_proba(data_test[:, n5, :].T)
                    auc_rf[n4, n5] = roc_auc_score(label_test, pred_rf[:, 1])

                for n5 in range(num_shuffle):
                    label_train = shuffle(label_train)
                    for n6 in range(T):
                        model_glm = LogisticRegression()
                        model_glm.fit(data_train[:, n6, :].T, label_train)
                        pred_glm = model_glm.predict_proba(data_test[:, n6, :].T)
                        auc_glm_shuffle[n4, n6, n5] = roc_auc_score(label_test, pred_glm[:, 1])

                        model_rf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=50, random_state=0)
                        model_rf.fit(data_train[:, n6, :].T, label_train)
                        pred_rf = model_rf.predict_proba(data_test[:, n6, :].T)
                        auc_rf_shuffle[n4, n6, n5] = roc_auc_score(label_test, pred_rf[:, 1])

            decode_trial_type_auc_local[decode_idx2trial_type[n2] + "_vs_" + decode_idx2trial_type[n3] + "_glm"] = auc_glm
            decode_trial_type_auc_local[decode_idx2trial_type[n2] + "_vs_" + decode_idx2trial_type[n3] + "_rf"] = auc_rf
            decode_trial_type_auc_local[decode_idx2trial_type[n2] + "_vs_" + decode_idx2trial_type[n3] + "_glm_shuffle"] = auc_glm_shuffle
            decode_trial_type_auc_local[decode_idx2trial_type[n2] + "_vs_" + decode_idx2trial_type[n3] + "_rf_shuffle"] = auc_rf_shuffle

    decode_trial_type_auc[n1] = decode_trial_type_auc_local

# Clear variables from the current namespace (you can manually remove unnecessary variables if needed)
del num_fold, num_shuffle, decode_idx2trial_type, decode_idx, n1, decode_trial_type_auc_local, n2, n3, data_temp_pos, data_temp_neg
del n4, num_trial_pos, num_trial_neg, auc_glm, auc_rf, auc_glm_shuffle, auc_rf_shuffle
del data_test, label_test, data_train, label_train, n5, model_glm, pred_glm, auc_temp, auc_glm, model_rf, pred_rf, n6